# RNN 의 구조

- 중요 정보 1 : 맥락이 중요하다 (Context)
- 중요 정보 2 : 시퀀스 순서가 중요하다 [the, movie, is ,fun]

## RNN이 중요한 이유

- 시퀀스의 컨텍스트 정보(단어 간의 순서 및 상호작용)

```python
# tensorflow

model = models.Sequential()
model.add(layers.Embedding(max_features, 64, input_length=max_len))
model.add(layers.SimpleRNN(32, activation='tanh', return_sequences=False))
model.add(layers.Dense(16, activation='tanh'))
model.add(layers.Dense(2, activation = 'softmax'))
model.summary()
```

## 시작 토큰 및 종료 토큰

- 시작 토큰 : 각 시퀀스의 시작점을 알림
- 종료 토큰 : 각 시퀀스의 종료점을 알림

# RNN 순서 간단 요약

1. 각 단어를 임베딩 벡터로 변환
    
    ```python
    model.add(layers.Embedding(max_features, 64, input_length=max_len))
    ```
    
<img src="..\..\img\_07_RNN\rnn3.png" width="1000">
    
2. RNN 안에서의 움직임
    
    ```python
    model.add(layers.SimpleRNN(32, activation='tanh', return_sequences=False))
    ```
    
    - time step 0 의 단계

        <img src="..\..\img\_07_RNN\rnn4.png" width="1000">

        
        1. 임베딩 으로 변환된 0 번째 단어를 RNN 층으로 연산
            1. RNN층의 각 노드와 연산한다. $h_0, h_1 … h_j$
        2. 각 임베딩 벡터와 연산 된 결과를 hidden state 0 벡터안의 요소로 넣는다
            
            $$
            h_{\{time \space step\}, \{node \space number \}}
            $$
            
            $\text{hidden state vec}_0 = (h_{0,1}, h_{0,1}, … h_{0,j})$ - 0 번째 time step 의 히든 스테이트 벡터
            
    - time step 1 의 단계

        <img src="..\..\img\_07_RNN\rnn5.png" widht="1000">

        
        1. 다음 인덱스의 단어 임베딩과 이전 층의 hidden state 를 concat 한다.
        2. 만약 movie 의 임베딩(64 차원)이 $[e_{movie0},e_{movie1},e_{movie2}...e_{movie63}]$ 라면
            
            RNN 에 입력값은 아래와 같다. 즉 입력값의 차원은 (64 + j) 가 된다.
            
    
    $$
    [e_{movie0},e_{movie1},e_{movie2}...e_{movie63},h_{0,1}, h_{0,1}, … h_{0,j} ]
    $$
    
    - time step 2 이상부터는 time step 1 과정과 동일하다 그리고 마지막 step 에서 종결되고, 그 후 dense layer 쪽으로 출력된다 (FNN)
    - 이후 문제에 따라 정답 라벨을 예측한다
- 주의 할 점 : 단어 하나가 들어가는 것이 아니라 시퀀스 단위로 입력된다.

# Return_Sequence 파라미터

<img src="../../img/_07_RNN/rnn6.png" width="500">

In [17]:
import numpy as np
from tensorflow.keras import layers

embeded_sequence = np.array([
    [ # 시퀀스의 예시
        [0.1, 4.2, 1.5, 1.1, 2.8], # the
        [1.0, 3.1, 2.5, 0.7, 1.1], # movie
        [0.3, 2.1, 1.5, 2.1, 0.1], # is
        [2.2, 1.4, 0.5, 0.9, 1.1]  # fun
    ]], dtype=np.float32)

rnn_rs_true = layers.SimpleRNN(3, return_sequences=True, return_state=True)
rnn_rs_false = layers.SimpleRNN(3, return_sequences=False, return_state=True)

print("마지막 층의 hidden state vector를 보여준다")
hidden_state, _ = rnn_rs_false(embeded_sequence)
print(hidden_state, end="\n\n")

hidden_states, _ = rnn_rs_true(embeded_sequence)
print("모든 층의 hiddne state vector를 보여준다")
print(hidden_states)

마지막 층의 hidden state vector를 보여준다
tf.Tensor([[-0.26363745 -0.9214386  -0.96820176]], shape=(1, 3), dtype=float32)

모든 층의 hiddne state vector를 보여준다
tf.Tensor(
[[[0.9727123  0.9999898  0.0140559 ]
  [0.99980825 0.99822253 0.90205467]
  [0.9998661  0.990903   0.70682365]
  [0.9992651  0.96756256 0.30586708]]], shape=(1, 4, 3), dtype=float32)


- False 시에 : 마지막 히든 스테이트 벡터만을 다음층에 넘겨준다.
    
    ```python
    modelno = models.Sequential()
    modelno.add(layers.Embedding(max_features, 64, input_length=max_len))
    modelno.add(layers.SimpleRNN(32, return_sequences=False)) # 32 노드의 RNN 층 h0 ~ h31 까지 존재
    
    # 모든 시퀀스를 돌고나서 나온 마지막 hidden state vector 31을 flatten 하여 Dense 층과 연결한다.
    modelno.add(layers.Flatten())
    modelno.add(layers.Dense(16, activation='tanh'))
    modelno.add(layers.Dense(2, activation = 'softmax'))
    modelno.summary()
    ```
    
- True 시에 : 모든 히든 스테이트 벡터를 다음층에 넘겨준다. (Flatten 해준 후에 넘겨줘서 다음 덴스층에 연결됨)
    
    ```python
    model = models.Sequential()
    model.add(layers.Embedding(max_features, 64, input_length=max_len))
    model.add(layers.SimpleRNN(32, return_sequences=True)) # 32 노드의 RNN 층 h0 ~ h31 까지 존재
    
    # 모든 시퀀스를 돌고나서 나온 마지막 hidden state vector 31을 flatten 하여 Dense 층과 연결한다.
    model.add(layers.Flatten())
    model.add(layers.Dense(16, activation='tanh'))
    model.add(layers.Dense(2, activation = 'softmax'))
    model.summary()
    ```

## 출력 형태

False = (batch_size, output_dim)

True = (batch_size, time steps,output_dim)

# 모든 Hidden state 를 보낸 후에 mean

```python
main_input = Input(shape=(max_len,), dtype='int32')
# 단어의 임베딩 64차원
x = Embedding(output_dim=64, input_dim=max_features, input_length=max_len)(main_input)

# RNN 을 거치면서 32차원으로 줄어든다.
# True 로 줬기 때문에 모든 히든스테이트를 다음층으로 보낸다.
RNN_out = SimpleRNN(32, return_sequences=True)(x)
# 여기서의 출력은 (batchsize, hidden State ,32)

# 각 hidden state 위치 별 32 개의 값의 평균을 구해 넣는다
# 즉, 각 단어의 32 개의 히든 스테이트의각 노드의 평균

out = Lambda(lambda x: tf.math.reduce_mean(x, axis=1))(RNN_out)
# 출력 겂은 (batch size, 32)
```

각 배치 별 한 문서라고 가정할 때

batch 1: 문서 1 = [ 문서에 대하여 RNN층의 hidden state 의 0 번째 , 1 번째, 2 번째 … , 31 번째 평균]

batch 2: 문서 2 = [ 문서에 대하여 RNN층의 hidden state 의 0 번째 , 1 번째, 2 번째 … , 31 번째 평균]

… 모든 배치에서 동일

## 의미 및 도식화

<img src="../../img/_07_RNN/rnn7.png" width="1000">

- 모든 문서(각 각의 배치안의 인덱스)들을 벡터화 하게 된다

# 두개 이상의 RNN 층인 stacked RNN 층일경우 Return_Sequences 설정은 무조건 TRUE 로 해야 한다